In [ ]:
'''
File name: project.ipynb
Author: ...Jose, Mohamed Ndoye, Raphael Strebel
Date created: 03/11/2019
Date last modified: ...
Python Version: 3.7.4
''';

<a id="up"></a>
# Food Inspections in Chicago

 - [Load Databases](#load-databases)
 - [Complete Datasets](#complete-datasets)
 - [Maps](#maps)
 - [Basic Statistics](#basic-stats)

In [ ]:
# useful : https://www.sustainabilist.com/blog/chicago-data-analysis-a-internship-project

import pandas as pd
import numpy as np
# TODO : Add to README "download libraries geopandas, vincent,..." with a short description to explain its use
import geopandas as gpd

import vincent
vincent.core.initialize_notebook() 

from utils import constants as cst
from utils import clean_database
from utils import areas_handler
from utils import chicago_map_handler as maps


import folium
import json
import math

# Set auto-reload 
%load_ext autoreload
%autoreload 2

<a id = 'load-databases'></a>
## Load Databases

In this section we load and clean the databases.

[Table of Contents](#up)

In [ ]:
# Load the areas dataframe 
areas_DF = gpd.read_file(cst.AREAS_PATH)

# Clean the dataframe
areas_DF = clean_database.clean_areas_df(areas_DF)
areas_DF.head()

In [ ]:
# Load the food inspections dataframe
food_inspections_DF = pd.read_csv(cst.FOOD_INSPECTIONS_PATH, sep = ',', header = 0, 
                   names = cst.FOOD_INSPECTIONS_COL_NAMES, index_col = None, error_bad_lines=False
                   )

# Clean the dataframe
food_inspections_DF = clean_database.clean_food_inspections_df(food_inspections_DF, areas_DF)
food_inspections_DF.head()

In [ ]:
# Load the socio-economic indicators dataframe
socio_economic_DF = pd.read_csv(cst.SOCIO_ECONOMIC_INDICATORS_PATH, sep = ',', header = 0, 
                   names = cst.SOCIO_ECONOMIC_COL_NAMES, index_col = None, error_bad_lines=False
                   )

# Clean the dataframe
socio_economic_DF = clean_database.clean_socio_economic_df(socio_economic_DF)
socio_economic_DF.head()

In [ ]:
# Load the life expectancy dataframe
life_expectancy_DF = pd.read_csv(cst.LIFE_EXPECTANCY_PATH, sep = ',', header = 0, 
                   names = cst.LIFE_EXPECTANCY_COL_NAMES, index_col = None, error_bad_lines=False
                   )

# Clean the dataframe
life_expectancy_DF = clean_database.clean_socio_economic_df(life_expectancy_DF)
life_expectancy_DF.head()

## <a id = 'complete-datasets'></a>
## Complete Datasets

### 2 problems : 
1. we only have the area name for the life exp. and the socio-eco DFs -> find the regions in sequence of lat/lng pairs that corresponds to the boundaries of an area. Then we can determine the region of the facility of the food_inspections dataframe and work only with the regions for the rest of the project (thoughts ?).
2. some entries in food_inspections_DF have no lat/lng pair -> must find it given their address

[Table of Contents](#up)

In [ ]:
# merge socio-economic and life expectancy df's on the area number and names
socio_life_merged_DF = socio_economic_DF.merge(life_expectancy_DF, how="left", on=["community_area_num", "community_area_name"])# [["community_area_num", "community_area_name"]]

In [ ]:
socio_life_merged_DF.head()

In [ ]:
# filter locations where lng/lat are unknown
food_unknown_loc = food_inspections_DF[food_inspections_DF['lat'].isna()]

#consider only those locations that are actually in the city of Chicago (or unknown) 
food_unknown_loc = food_unknown_loc[(food_unknown_loc['city'].str.lower() == 'chicago') | (food_unknown_loc['city'].isna())]

In [ ]:
# Get unknown locations 
unknown_locations = areas_handler.get_unknown_locations(food_unknown_loc)

In [ ]:
unknown_locations.head()

In [ ]:
#check the locations not found by OpenStreetMaps
unknown_locations[pd.isnull(unknown_locations['lat'])]

In [ ]:
# Display the Chicago areas
# This map contains additional layers to visually check if the found locations are actually within the borders of the city
map_chicago = maps.create_chicago_map()
map_chicago = maps.add_locations(map_chicago, unknown_locations, food_inspections_DF) 
map_chicago

**Description of the map**

On the map above we can see the city of Chicago and the region where the facilities of the inspections presented in the food_inspections dataset are located. In the following, we will focus ang get some insights on that area.

In [ ]:
# Use unknonwn_locations to fill lat and lng in the original dataframe food_inspections_DF

food_unknown_loc = food_unknown_loc.reset_index().merge(unknown_locations, on="address", how='left').set_index('index')
food_unknown_loc = food_unknown_loc.drop(['lat_x', 'lng_x'], axis = 1)
food_unknown_loc = food_unknown_loc.rename(columns={'lat_y':'lat','lng_y':'lng'})

food_inspections_DF.update(food_unknown_loc)
food_inspections_DF[food_inspections_DF['lat'].isna()]


In [ ]:
# Resolve are numbers and delete unknown areas
# this takes a while

food_inspections_DF = areas_handler.get_locations_with_area(food_inspections_DF, areas_DF)
print("Number of locations: " + str(food_inspections_DF.shape[0]))
#drop locations not in the city of chicago 
food_inspections_DF = food_inspections_DF.dropna(subset=[cst.AREA_NUM])
print("Number of locations in the city of chicaco: " + str(food_inspections_DF.shape[0]))
food_inspections_DF[cst.AREA_NUM] = food_inspections_DF[cst.AREA_NUM].astype(int)



<a id = 'maps'></a>
## Maps

We visualize some of the data using folium to find connections

[Table of Contents](#up)

### Number of inspections map

In [ ]:
# TODO there is a SettingWithCopyWarning: 
# A value is trying to be set on a copy of a slice from a DataFrame.
# Try using .loc[row_indexer,col_indexer] = value instead

#create new dataframe with number of inspections per area
inspection_counts = food_inspections_DF[cst.AREA_NUM].value_counts().to_frame()
inspection_counts.reset_index(level=0, inplace=True)
inspection_counts.columns=[cst.AREA_NUM,'num_inspections']
inspection_counts[cst.AREA_NUM] = inspection_counts[cst.AREA_NUM].astype(str)
inspection_counts.sort_values('num_inspections');

In [ ]:
heatmap_chicago = maps.heat_map(inspection_counts, "Inspections", cst.AREA_NUM, 'num_inspections')
heatmap_chicago

**Description of the map**

The heatmap above shows the number of inspections on the area we are focusing on and determined before. We can see that the more we approach the city center of Chicago, the higher the number of inspections.

### Number of Establishments

In [ ]:
count_DF = food_inspections_DF.copy()
count_DF = count_DF.drop_duplicates(subset=['license_num'])
count_ser = count_DF[cst.AREA_NUM].value_counts().to_frame()
count_ser.reset_index(level=0, inplace=True)
count_ser.columns=[cst.AREA_NUM,'num_establishments']
count_ser[cst.AREA_NUM] = count_ser[cst.AREA_NUM].astype(str)
count_ser.sort_values('num_establishments');

In [ ]:
heatmap_chicago = maps.heat_map(count_ser, "Number of Establishments", cst.AREA_NUM, 'num_establishments')
heatmap_chicago

**Description of the map**

The heatmap above shows the number of establishments on the area we want to analyze. Same as before, we observe that the more we approach the city center, the higher the number of establishments. Indeed, the number of inspections is correlated with the number of establishments; the more establishments we have on a given area, the higher the inspections will be on that area. 

### Average risk per inspection for each area

In [ ]:
def risk_to_num(x):
    if x == 'Risk 3 (Low)':
        return 1
    if x == 'Risk 2 (Medium)':
        return 2
    if x == 'Risk 1 (High)':
        return 3
    if x == 'All':
        return 2
    else:
        return x

risk_DF = food_inspections_DF.copy()
risk_DF = risk_DF.dropna(subset=['risk'])
risk_DF['risk'] = risk_DF['risk'].apply(lambda x : risk_to_num(x)).astype(int)
risk_ser = risk_DF.groupby(cst.AREA_NUM)['risk'].mean().to_frame()
risk_ser.reset_index(level=0, inplace=True)
risk_ser
risk_ser[cst.AREA_NUM] = risk_ser[cst.AREA_NUM].astype(str)


In [ ]:
heatmap_chicago = maps.heat_map(risk_ser, "Average risk", cst.AREA_NUM, 'risk')
heatmap_chicago

**Description of the map**

The heatmap above shows the average risk of the establishments for each community area. The risk of an establishment shows how it could potentially affect the public's health with 1 being the highest and 3 the lowest. Furthermore, **high risk establishments are inspected more frequently that low risk establishments**.


### Number of inspections per establishment

In [ ]:
#inspections_per_est = inspection_counts.merge(count_ser, left_on=cst.AREA_NUM, right_on=cst.AREA_NUM).drop(['index'], axis = 1)
inspections_per_est = inspection_counts.merge(count_ser, left_on=cst.AREA_NUM, right_on=cst.AREA_NUM)
inspections_per_est['inspections_per_establishment'] = inspections_per_est.apply(lambda x : x.num_inspections/x.num_establishments, axis = 1)

In [ ]:
heatmap_chicago = maps.heat_map(inspections_per_est, "Number of Inspections per Establishment", cst.AREA_NUM, 'inspections_per_establishment')
heatmap_chicago

**Description of the map**

The heatmap above shows the number of inspections per establishment and as highlighted before, we can see that the establishments with a high number of inspections present high average risks. 

### Inspections by date

In [ ]:
inspection_by_year = food_inspections_DF.copy()
inspection_by_year['inspection_date'] = pd.DatetimeIndex(inspection_by_year['inspection_date']).year
inspection_by_grouped =  inspection_by_year.groupby([cst.AREA_NUM, 'inspection_date']).size().to_frame()
inspection_by_grouped = inspection_by_grouped.unstack()
inspection_by_grouped.columns = inspection_by_grouped.columns.get_level_values(1)
inspection_by_grouped.reset_index(level=0, inplace=True)
#inspection_by_grouped = inspection_by_grouped.drop(columns='inspection_date')
inspection_by_grouped

In [ ]:
# i have no idea why this does not work...i honsetly thinkt that the plugin is simply broken
# fixed something by adding 'import folium.plugins' in chicago_map_handler.py but still can't see the heatmap...
maps.timed_heatmap(inspection_by_grouped, areas_DF)

### Violations

Each establishment can receive a violation number between 1-44 or 70 and the number is followed by a specific description of the findings that caused the violation to be issued. The higher the number, the better the description. Establishments collecting only high numbers might probably pass whereas the others might probably fail.

An inspection of an establishment can pass, pass with conditions or fail depending on these numbers. The 'pass' condition is given when the establishment were found to have no critical or serious violations. Establishments that received a 'pass with conditions' were found to have critical or serious violations but these violations were corrected during the inspection. Finally, the 'fail' condition is issued when the establishment were found to have critical or serious violations that were not correctable during the inspection.


### Socioeconomic stuff

In [72]:
#Select one of the following columns for the heatmap
socio_life_merged_DF[cst.AREA_NUM] = socio_life_merged_DF[cst.AREA_NUM].astype(str)
socio_life_merged_DF.columns

Index(['community_area_num', 'community_area_name', 'housing_crowded_perc',
       'housholds_below_poverty_perc', 'aged_16_or_more_unemployed_perc',
       'aged_25_or_more_without_high_school_diploma_perc',
       'aged_under_18_or_over_64_perc', 'per_capita_income', 'hardship_idx',
       'life_exp_1990', 'lower_95_perc_CI_1990', 'upper_95_perc_CI_1990',
       'life_exp_2000', 'lower_95_perc_CI_2000', 'upper_95_perc_CI_2000',
       'life_exp_2010', 'lower_95_perc_CI_2010', 'upper_95_perc_CI_2010'],
      dtype='object')

In [ ]:
heatmap_chicago = maps.heat_map(socio_life_merged_DF, "Life expectancy 2010",cst.AREA_NUM ,'life_exp_2010')
heatmap_chicago

<a id = 'basic-stats'></a>
## Basic Statistics

We report some statistics on the various dataframes.

[Table of Contents](#up)

In [ ]:
corr = socio_life_merged_DF[cst.SOCIOECONOMIC_METRICS].corr()
corr

In [ ]:
bad_metrics = set(['housing_crowded_perc', 'housholds_below_poverty_perc', 'aged_16_or_more_unemployed_perc', 
               'aged_25_or_more_without_high_school_diploma_perc', 'hardship_idx', 'aged_under_18_or_over_64_perc'])
good_metrics = set(['per_capita_income', 'life_exp_2010' ])
sign_kept = True

for c1 in cst.SOCIOECONOMIC_METRICS:
    for c2 in cst.SOCIOECONOMIC_METRICS:
        if (c1 in bad_metrics and c2 in bad_metrics) or (c1 in good_metrics and c2 in good_metrics):
            if corr[c1][c2] < 0:
                sign_kept = False
        elif (c1 in bad_metrics and c2 in good_metrics) or (c1 in good_metrics and c2 in bad_metrics):
            if corr[c1][c2] > 0:
                sign_kept = False
print(sign_kept)

In [ ]:
#set correlation between each variable and itself to None in order to ignore it later
for c in corr.columns:
    corr[c][c] = None 
    
corrmax =pd.DataFrame(corr.idxmax()).rename({0: 'Strongest positive correlation'}, axis = 1)
corrmax['Correlation value'] = corr.max()
corrmax

In [ ]:
corrmin =pd.DataFrame(corr.idxmin()).rename({0: 'Strongest negative correlation'}, axis = 1)
corrmin['Correlation value'] = corr.min()
corrmin

Of the above correlations, we notice certain things: Firstly, we can classify the indicators between good (life expectancy and per capita income) and bad (percentage of crowded houses, percentage of below porverty households, percentage of over 16 unemployed people, percentage fo over 25 people without a high school diploma, the hardship index, and the percentage of people under 18 and over 64), and the correlation between indicators either both good or both bad will always be positive, whereas the correlation between a good and a bad indicator will always be negative. 

We also notice that the percentage of people under 18 or over 64 is a strong negative indicator: it is more negatively correlated to per capita income than, for example, the percentage of houses living below the poverty line. 

It is indeed quite surprising that per capita average income is not more correlated to the percentage of houses living below the poverty line (correlation is -0.56). We plot the 2 metrics in order to see this:

One reason the linear correlation is so low is that the relationship is exponential. Also, the top 5 highest per capita neighbourhoods are not in the top 15 lowest poor households percentage. TODO: why does this happen??'?!!! where (very downtown). What are some other indicators in this 'mixed' (rich and poor people) neighbourhoods?? This is a cool direction to go in i think

In [ ]:
scatter = vincent.Scatter(socio_life_merged_DF[['per_capita_income','housholds_below_poverty_perc']], iter_idx = 'housholds_below_poverty_perc')

In [ ]:
scatter.axis_titles(x='Percentage of households below the poverty line', y='per_capita_income')


TODO: You can not click in the above plot. It'd be cool to be able to click and see the name of a neighbourhood. How can we do this with vincent? I kknow how with plotly but they said to use vincent. 

CONTINUATION: finish socioeconomic things (average, stard deviation, max min. How many people live in poor areas, how many people live in bad areas, etc.). Maybe classify areas in 4 manually. Plot where al this is.

Once socioeconomic things are done, let's look at food things (inspections: where are they happening)?

Finally, look at correlation between the 2.
